## This is run on memdisk

```
# create a 32GB tmpfs
sudo mkdir /mnt/mnttmp
sudo mount -t tmpfs -o size=32G tmpfs /mnt/mnttmp
sudo truncate -s 32G /mnt/mnttmp/loopfile

# create and format loop device
sudo losetup -f /mnt/mnttmp/loopfile
sudo losetup -j /mnt/mnttmp/loopfile
# output will say which loop device was used, e.g. loop0

sudo mkfs.xfs /dev/loop0
sudo rm -rf ~/ceph/build/dev/ ~/ceph/build/out/
sudo mkdir ~/ceph/build/dev/
sudo mount -t xfs /dev/loop0 ~/ceph/build/dev
sudo chown $USER:$USER ~/src/ceph/build/dev

[osd]
 bluestore block create = true
 bluestore block size = 32G

If you're using vstart.sh, you can pass these options via -o.
```

system:

`$ lsb_release -a` 
`$ uname -a` 
```
Ubuntu 18.04.1 LTS
Linux c220g5-110527 4.15.0-88-generic #88-Ubuntu SMP Tue Feb 11 20:11:34 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux
GNU/Linux
```
`$ sudo smartctl -a /dev/sda` 
```
Device Model:     INTEL SSDSC2BB480G7K
Serial Number:    PHDV724404VV480BGN
LU WWN Device Id: 5 5cd2e4 14e0dd210
Firmware Version: N201CS01
User Capacity:    480,103,981,056 bytes [480 GB]
Sector Sizes:     512 bytes logical, 4096 bytes physical
Rotation Rate:    Solid State Device
Form Factor:      2.5 inches
Device is:        Not in smartctl database [for details use: -P showall]
ATA Version is:   ACS-3 T13/2161-D revision 5
SATA Version is:  SATA 3.1, 6.0 Gb/s (current: 6.0 Gb/s)
Local Time is:    Wed Jun  3 12:20:23 2020 CDT
SMART support is: Available - device has SMART capability.
SMART support is: Enabled
```


ceph config:
```
    osd_op_num_shards = 8
    osd_op_num_threads_per_shard = 2
    enable_throttle = false
    enable_codel = false
    enable_batch_bound = false
    kv_queue_upper_bound_size = 30
    bluestore_throttle_bytes = 100000000
    bluestore_throttle_deferred_bytes = 100000000
    bluestore_throttle_cost_per_io = 0
    bluestore_throttle_cost_per_io_hdd = 100000
    bluestore_throttle_cost_per_io_ssd = 100000
    bdev_block_size = 4096
    bluestore_min_alloc_size = 4096
    bluestore_min_alloc_size_hdd = 65536
    bluestore_min_alloc_size_ssd =  4096
    bluestore_max_alloc_size = 0
    bluestore_prefer_deferred_size = 0
    bluestore_prefer_deferred_size_hdd = 0
    bluestore_prefer_deferred_size_ssd = 0
``` 
simple writes dominate

workload:
```
[global]
ioengine=rbd
clientname=admin
pool=mybench
rbdname=image1
rw=randwrite
bs=4096
;size=83886080000
time_based=1
runtime=300
direct = 1
;write_iops_log

[fio-write]
iodepth=48

```

Start Ceph cluster:\
`sudo MON=1 OSD=1 MDS=0 ../src/vstart.sh -n -b -x -l  -o 'bluestore fsck on mkfs = false' -o 'bluestore fsck on mount = false' -o 'bluestore fsck on umount = false' -o 'bluestore block db path = ' -o 'bluestore block wal path = ' -o 'bluestore block wal create = false' -o 'bluestore block db create = false' -o 'bluestore block create = true' -o 'bluestore block size = 32G' --without-dashboard`

OSD is is sharing SSD with monitor(has filesystem on it)

rbd info:

In [1]:
f = open('dump_rbd_info.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

rbd image 'image1':
	size 1 GiB in 256 objects
	order 22 (4 MiB objects)
	snapshot_count: 0
	id: 101f8af1ff0a
	block_name_prefix: rbd_data.101f8af1ff0a
	format: 2
	features: layering, exclusive-lock, object-map, fast-diff, deep-flatten
	op_features: 
	flags: 
	create_timestamp: Thu Jun  4 15:01:35 2020
	access_timestamp: Thu Jun  4 15:01:35 2020
	modify_timestamp: Thu Jun  4 15:26:05 2020

